In [1]:
# -*- coding: utf-8 -*-

import config
from tools import datacleaner as dtclean
from tools import dataexplore as dtexp
from view import data_visualization as dtview
import pandas as pd
import numpy as np
from fbprophet import Prophet
from evaluation import forecast_metrics as fm
import seaborn as sns
import plotly.graph_objs as go
import plotly.tools
import matplotlib.pyplot as plt
# Offline mode
import plotly.offline as py
from plotly.offline import init_notebook_mode, iplot
import warnings
import dask.dataframe as dd
import dask as dsk

In [11]:
pd.options.display.max_rows = 10
path = getattr(config,'csvpath','default_value')


In [12]:
df_lojas_vendas_caixa = dd.read_csv(path+'/FDATA_tienda_ventas_caja.csv')

df_lojas_vendas_caixa.head()

,tienda,caja,fecha,fechaHoraInicio,fechaHoraFim,cantidad,numero,tickets,importe
0,37,7,2020-05-09 00:00:00.000,2020-05-09 19:00:00.000,2020-05-09 19:15:00.000,13.0,9.0,8.0,6.98
1,37,7,2020-05-09 00:00:00.000,2020-05-09 19:15:00.000,2020-05-09 19:30:00.000,14.0,11.0,7.0,8.25
2,37,7,2020-05-09 00:00:00.000,2020-05-09 19:30:00.000,2020-05-09 19:45:00.000,9.0,6.0,1.0,6.05
3,37,7,2020-05-09 00:00:00.000,2020-05-09 19:45:00.000,2020-05-09 20:00:00.000,22.0,16.0,7.0,19.10
4,37,7,2020-05-09 00:00:00.000,2020-05-09 20:00:00.000,2020-05-09 20:15:00.000,6.0,4.0,2.0,3.66
